In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()


split_length = .125 # miles

In [2]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

In [3]:
# project geodatabases
dirPrj = r'G:\Data\Observatory\Transportation\RTP\RTP_2023'
working_directory = os.getcwd()
intermediate_folder = os.path.join(working_directory, "intermediate")

dfProjects = pd.DataFrame([
    ['Roadway','points',r'RTP_2023_Roadway.gdb\RTP_2023_Roadway_points'],
    ['Roadway','lines' ,r'RTP_2023_Roadway.gdb\RTP_2023_Roadway_lines' ],
    ['Transit','points',r'RTP_2023_Transit.gdb\RTP_2023_Transit_points'],
    ['Transit','lines' ,r'RTP_2023_Transit.gdb\RTP_2023_Transit_lines' ],
    ['AT'     ,'points',r'RTP_2023_AT.gdb\RTP_2023_AT_points'          ],
    ['AT'     ,'lines' ,r'RTP_2023_AT.gdb\RTP_2023_AT_lines'           ],
],columns=('prjGroup','prjType','prjFC'))

In [4]:
# Read the data into a Pandas dataframe
sdfPrj_Lines  = pd.DataFrame()
sdfPrj_Points = pd.DataFrame()


# Loop through the rows of the original dataframe
for index, row in dfProjects.iterrows():
    _sdf = pd.DataFrame.spatial.from_featureclass(os.path.join(dirPrj,row['prjFC']))
    #display(_sdf)
    _sdf[['GIS_ID','SHAPE']].spatial.to_featureclass(os.path.join(intermediate_folder,'process.gdb\\_prj_' + row['prjGroup'] + '_' + row['prjType']), sanitize_columns=False)


c:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-again-again\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-again-again\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-again-again

In [5]:
from IPython.display import clear_output, display

# A TOGGLE TO SET SO THAT RUNNING NOTEBOOK AGAIN THIS PROCESS ISN'T REPEATED IF NOT NEEDED, ie if this is set to 'False' don't run
bSplitSegs = True

if bSplitSegs:

    # Loop through the rows of the original dataframe
    for index, row in dfProjects.iterrows():

        _fcLayer = row['prjGroup'] + '_' + row['prjType']
        _fcIn  = os.path.join(intermediate_folder,'process.gdb\\_prj_' + _fcLayer)
        _fcSeg = _fcIn + '_segs'

        _fcRtpLinesSplitPointsTmp = _fcIn + '_split_points_tmp'
        _fcRtpLinesSegsTmp = _fcIn + '_segs_tmp'

        if row['prjType'] == 'lines':
            
            delIfExists(_fcSeg)

            bFirst = True

            # Create a search cursor for the feature class
            fields = ["OID@", "SHAPE@", "gis_id"]
            cursor = arcpy.da.SearchCursor(_fcIn, fields)

            # Get the total number of rows in the feature class
            count = arcpy.GetCount_management(_fcIn)[0]

            # need to loop through each polyline separately since there are overlapping projects and have to be treated separately
            # Loop through each polyline in the input feature class
            for i, row in enumerate(cursor):

                # Get the OID and shape of the current polyline
                oid      = row[0]
                polyline = row[1]
                gid      = row[2]
                
                # delete temp layers
                delIfExists(_fcRtpLinesSplitPointsTmp)
                delIfExists(_fcRtpLinesSegsTmp)


                # Generate points along the polyline at the specified interval
                arcpy.GeneratePointsAlongLines_management(polyline, _fcRtpLinesSplitPointsTmp, 'DISTANCE', Distance = str(split_length) + ' miles')
                
                # Split the polyline at the generated points
                arcpy.SplitLineAtPoint_management(polyline, _fcRtpLinesSplitPointsTmp, _fcRtpLinesSegsTmp, '5 feet')

                
                # Add a new field to the output feature class to store the original OID of the polyline
                arcpy.AddField_management(_fcRtpLinesSegsTmp, "gis_id", "TEXT")

                # Populate the ORIG_OID field with the OID of the original polyline
                with arcpy.da.UpdateCursor(_fcRtpLinesSegsTmp, ["gis_id"]) as update_cursor:
                    for update_row in update_cursor:
                        update_row[0] = gid
                        update_cursor.updateRow(update_row)

                if bFirst:
                    arcpy.management.Copy(_fcRtpLinesSegsTmp, _fcSeg)
                    bFirst = False
                else:
                    arcpy.management.Append(_fcRtpLinesSegsTmp, _fcSeg)

                # Update the progressor
                progress = "Processing polylines for " + _fcLayer + ". Done processing: " + str(i+1) + " of " + str(count)
                display(progress, display_id="progress")

                # Clear the previous progress message
                clear_output(wait=True)

            # Clean up the cursor and progressor
            del cursor

'Processing polylines for AT_lines. Done processing: 673 of 673'

Took 51m 48.6s

In [7]:

# Loop through the rows of the original dataframe
for index, row in dfProjects.iterrows():

    _fcLayer = row['prjGroup'] + '_' + row['prjType']
    _fcIn  = os.path.join(intermediate_folder,'process.gdb\\_prj_' + _fcLayer)

    _fcRtpLinesSplitPointsTmp = _fcIn + '_split_points_tmp'
    _fcRtpLinesSegsTmp = _fcIn + '_segs_tmp'

    if row['prjType'] == 'lines':
        # delete temp layers
        delIfExists(_fcRtpLinesSplitPointsTmp)
        delIfExists(_fcRtpLinesSegsTmp)